In [61]:
import pandas as pd
import numpy as np

In [62]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data/"

## Lectura de archivos

In [63]:
#### Lectura de archivos

#df_id: Módulo de identificación  
df_id = pd.read_excel(data_path+"CENSO 2022/Capítulo 1.xlsx",header = 0)
#df_ib: Módulo identificación básica  
df_ib = pd.read_excel(data_path+"CENSO 2022/Capítulo 2.xlsx",header = 0)
#df_po: Módulo de personal ocupado
df_po = pd.read_excel(data_path+"CENSO 2022/Capítulo 3.xlsx",header = 0)
#df_cg: Módulo de costos y gastos
df_cg = pd.read_excel(data_path+"CENSO 2022/Capítulo 4.xlsx",header = 0)
#df_ing: Módulo de ingresos
df_ing = pd.read_excel(data_path+"CENSO 2022/Capítulo 5.xlsx",header = 0)
# df_res: Resultados de la entrevista
df_res = pd.read_excel(data_path+"CENSO 2022/CE_REGISTROS_COLA_STATUS.xlsx",header = 0)

C:\Users\carlo\AppData\Roaming\Python\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [64]:
print("Dimensiones df resultados entrevista:",df_res.shape, "\n")
print("Dimensiones df identificación:",df_id.shape, "\n")
print("Dimensiones df info básica:",df_ib.shape, "\n")
print("Dimensiones df personal ocupado:",df_po.shape, "\n")
print("Dimensiones df costos y gastos:",df_cg.shape, "\n")
print("Dimensiones df ingresos:",df_ing.shape, "\n")

Dimensiones df resultados entrevista: (3888, 32) 

Dimensiones df identificación: (3514, 49) 

Dimensiones df info básica: (3514, 99) 

Dimensiones df personal ocupado: (3514, 81) 

Dimensiones df costos y gastos: (3514, 72) 

Dimensiones df ingresos: (3514, 26) 



## Filtrado de la información
1. Quedarse con las encuestas:  
    Status = Valido  
    Estado = Completa

In [65]:
df_res = df_res[df_res['STATUS']=='VALIDO']
df_res = df_res[df_res['ESTADO']=='Completa']

In [66]:
df = pd.merge(df_res, df_id, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_ib, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_po, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_cg, on='UID_ENCUESTA', how='left')
df = pd.merge(df, df_ing, on='UID_ENCUESTA', how='left')

C:\Users\carlo\AppData\Local\Temp\ipykernel_6588\1150504175.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ID_x', 'ID_ENCUESTA_x', 'IMEI_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_po, on='UID_ENCUESTA', how='left')
C:\Users\carlo\AppData\Local\Temp\ipykernel_6588\1150504175.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'USUARIO_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_cg, on='UID_ENCUESTA', how='left')
C:\Users\carlo\AppData\Local\Temp\ipykernel_6588\1150504175.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ID_x', 'FECHACREACION_x', 'ID_ENCUESTA_x', 'IMEI_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_ing, on='UID_ENCUESTA', how='left')


## Ajuste variables

In [67]:
# Definición labels y diccionarios
dic_depto = {5:"Antioquia",8:"Atlántico",11:"Bogotá, D. C.",13:"Bolívar",15:"Boyacá",17:"Caldas",18:"Caquetá",
             19:"Cauca",20:"Cesar",23:"Córdoba",25:"Cundinamarca",27:"Chocó",41:"Huila",44:"La Guajira",47:"Magdalena",
             50:"Meta",52:"Nariño",54:"Norte De Santander",63:"Quindío",66:"Risaralda",68:"Santander",70:"Sucre",73:"Tolima",
             76:"Valle Del Cauca",88:"San Andrés"}

dic_sector = {2:"Industria",3:"Comercio",4:"Servicios"}

dic_rut = {1:"Sí",0:"No"}

dic_g12 = {3:"Industria",
           4:"Construcción",
           5:"Comercio",
           6:"Transporte y almacenamiento",
           7:"Alojamiento/servicios de comida",
           8:"Información y comunicaciones",
           9:"Actividades inmobiliarias",
           10:"Educación",
           11:"Actividades de salud humana",
           12:"Actividades artísticas",
}

rel_reg_con= pd.read_excel(data_path + "Diccionario.xlsx", sheet_name="REG_CONT")
rel_ubica= pd.read_excel(data_path + "Diccionario.xlsx", sheet_name="UBICA")

### Ingresos

In [68]:
df['ING_TOTAL'].describe()

count    1.114000e+03
mean     1.078777e+08
std      6.747386e+08
min      0.000000e+00
25%      4.000000e+06
50%      1.500000e+07
75%      5.000000e+07
max      1.293199e+10
Name: ING_TOTAL, dtype: float64

In [69]:
#vars_ings = ["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN","ING_DON","ING_CONSTRUC","ING_ALQME"]
(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1)).value_counts()

True     1069
False     910
dtype: int64

In [70]:
ids = (df['UID_ENCUESTA'][((df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']]==99).any(axis = 1))]).tolist()

In [71]:
print("Número de NA's de la var. Ingreso cuando la suma de las individuales no coincide:")
df['ING_TOTAL'][~(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1))].isna().sum()
# 865 datos son NA y el resto pueden ser otro ingresos no contemplados al momento del Operativo

Número de NA's de la var. Ingreso cuando la suma de las individuales no coincide:


865

In [72]:
ids = df['UID_ENCUESTA'][~(df['ING_TOTAL'] == df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS']].sum(axis=1)) & ~(df['ING_TOTAL'].isna())]

In [73]:
df[["ING_IND","ING_MAQUIL","ING_MMTO","ING_COM","ING_SERV","ING_OTROPER","ING_FINAN",'ING_OTROS','ING_TOTAL']][df['UID_ENCUESTA'].isin(ids)]

,ING_IND,ING_MAQUIL,ING_MMTO,ING_COM,ING_SERV,ING_OTROPER,ING_FINAN,ING_OTROS,ING_TOTAL
14,0.0,0.0,0.0,99.0,10000000.0,100000.0,NaN,NaN,1.010000e+07
59,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
179,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
218,99.0,99.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
441,80000000.0,0.0,400000.0,3000000.0,99.0,0.0,NaN,NaN,8.340000e+07
444,99.0,0.0,0.0,99.0,20000000.0,0.0,NaN,NaN,2.000000e+07
513,99.0,0.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
516,99.0,0.0,99.0,99.0,99.0,99.0,NaN,NaN,0.000000e+00
608,99.0,0.0,0.0,99.0,99.0,0.0,NaN,NaN,0.000000e+00
928,0.0,0.0,0.0,0.0,2.0,0.0,NaN,NaN,1.000000e+10


In [74]:
np.sum([1,1,np.nan])

nan

### Costos y Gastos

#### Gastos

In [75]:
(df['GTOTAL_ANO'] == df[["GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]].sum(axis=1)).value_counts()

False    1144
True      835
dtype: int64

In [76]:
df[['GTOTAL_ANO',"GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]][(df[['GTOTAL_ANO',"GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]]==99).any(axis=1)]

,GTOTAL_ANO,GARRIEN_ANO,GENERGIA_ANO,GCOMUN_ANO,GACUED_ANO,GCOMB_ANO,GMANT_ANO,GOTROS_ANO


In [77]:
(df['GTOTAL']==99).value_counts()

False    1979
Name: GTOTAL, dtype: int64

In [78]:
print("Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:")
df['GTOTAL_ANO'][~(df['GTOTAL_ANO'] == df[["GARRIEN_ANO","GENERGIA_ANO","GCOMUN_ANO","GACUED_ANO","GCOMB_ANO","GMANT_ANO","GOTROS_ANO"]].sum(axis=1))].isna().sum()
# El total de datos que no coinciden son NA's

Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:


1144

In [79]:
(df['GTOTAL_ANOV'] == df[["GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]].sum(axis=1)).value_counts()

False    1726
True      253
dtype: int64

In [80]:
df[['GTOTAL_ANOV',"GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]][(df[['GTOTAL_ANOV',"GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]]==99).any(axis=1)]

,GTOTAL_ANOV,GARRIEN_ANOV,GENERGIA_ANOV,GCOMUN_ANOV,GACUED_ANOV,GCOMB_ANOV,GMANT_ANOV,GOTROS_ANOV


In [81]:
print("Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:")
df['GTOTAL_ANOV'][~(df['GTOTAL_ANOV'] == df[["GARRIEN_ANOV","GENERGIA_ANOV","GCOMUN_ANOV","GACUED_ANOV","GCOMB_ANOV","GMANT_ANOV","GOTROS_ANOV"]].sum(axis=1))].isna().sum()
# El total de datos que no coinciden son NA's

Número de NA's de la var. Gasto cuando la suma de las individuales no coincide:


1726

In [82]:
# De esta manera se suman los gastos de estableicmientos y viviendas en una sola variable
df['GASTOS'] =  df[['GTOTAL_ANO','GTOTAL_ANOV']].sum(axis=1)
# Dejar NA's en función de las variables originales

In [83]:
df.loc[(df['GTOTAL_ANOV'].isna()) & (df['GTOTAL_ANO'].isna()), 'GASTOS'] = np.nan

#### Costos

In [84]:
(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1)).value_counts()

True     1070
False     909
dtype: int64

In [85]:
df[['COSTO_TOTAL','COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']][(df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']]==99).any(axis=1)]

,COSTO_TOTAL,COSTO_FINAN,COSTO_HONO,COSTO_MAQ,COSTO_MMTO,COSTO_MPE,COSTO_PUB,COSTO_SERV,COSTO_TRANS,COSTO_VTA
59,0.0,0.0,99.0,99.0,0.0,99.0,99.0,99.0,99.0,99.0
192,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0
513,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,0.0,99.0
515,0.0,0.0,0.0,99.0,0.0,99.0,0.0,0.0,99.0,99.0
608,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,99.0,99.0
609,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0,99.0
653,0.0,0.0,0.0,0.0,99.0,99.0,0.0,0.0,99.0,99.0
844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0
1116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,99.0
1146,9000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000000.0,99.0


In [86]:
print("Número de NA's de la var. Costo cuando la suma de las individuales no coincide:")
df['COSTO_TOTAL'][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))].isna().sum()
# 866 de 909 datos que no coinciden son NA's

Número de NA's de la var. Costo cuando la suma de las individuales no coincide:


866

In [87]:
(df['COSTO_TOTAL'][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))]).value_counts()

0.0           37
9000000.0      1
22000000.0     1
800000.0       1
3000000.0      1
96.0           1
600000.0       1
Name: COSTO_TOTAL, dtype: int64

In [88]:
(df[['COSTO_TOTAL','COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']][~(df['COSTO_TOTAL'] == df[['COSTO_FINAN','COSTO_HONO','COSTO_MAQ','COSTO_MMTO','COSTO_MPE','COSTO_PUB','COSTO_SERV','COSTO_TRANS','COSTO_VTA']].sum(axis=1))])
# El valor restante de valores que no coinciden se debe a los 99 que presentan algunas de las variables individuales

,COSTO_TOTAL,COSTO_FINAN,COSTO_HONO,COSTO_MAQ,COSTO_MMTO,COSTO_MPE,COSTO_PUB,COSTO_SERV,COSTO_TRANS,COSTO_VTA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0
1976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Construcción de la variables de costos y gastos agregados

In [89]:
df['CYG'] = df[['COSTO_TOTAL','GASTOS']].sum(axis=1)

In [90]:
df[['COSTO_TOTAL','GASTOS','CYG']]

,COSTO_TOTAL,GASTOS,CYG
0,NaN,NaN,0.0
1,NaN,NaN,0.0
2,60000000.0,168720000.0,228720000.0
3,84000000.0,15840000.0,99840000.0
4,NaN,NaN,0.0
...,...,...,...
1974,12000000.0,1110000.0,13110000.0
1975,1000000.0,2400000.0,3400000.0
1976,NaN,NaN,0.0
1977,163704520.0,48840000.0,212544520.0


In [91]:
df.loc[(df['GASTOS'].isna()) & (df['COSTO_TOTAL'].isna()), 'CYG'] = np.nan

### Personal

In [92]:
df['POTTOT'].describe()

count    1130.000000
mean        3.197345
std        11.211273
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       322.000000
Name: POTTOT, dtype: float64

### Remuneración Total

In [93]:
df['UID_ENCUESTA'][(df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP']]==99).any(axis=1)]

35          70
399    4807221
Name: UID_ENCUESTA, dtype: int64

In [94]:
df['REMUNERACION_TOTAL'] = (df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP']].replace(99,np.nan).sum(axis=1))*df['MESOPA']

In [95]:
df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']][(df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']]==99).any(axis=1)]

,SALFIJOH,SALFIJOM,SALPERMH,SALPERMM,SALPOHTEMP,SALPOMTEMP,MESOPA
35,0.0,0.0,99.0,0.0,0.0,0.0,1.0
399,99.0,0.0,0.0,99.0,0.0,0.0,12.0


In [96]:
df['REMUNERACION_TOTAL'].describe()

count    1.155000e+03
mean     1.299048e+07
std      7.629155e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      6.500000e+06
max      1.582000e+09
Name: REMUNERACION_TOTAL, dtype: float64

### Departamento

In [97]:
df['DPTO'].value_counts()

91.0    984
13.0    641
11.0     51
Name: DPTO, dtype: int64

In [98]:
# Mantener sólo los departamentos que se encuentran en EMICRON
df['DPTO'].isin(dic_depto.keys()).value_counts()

False    1287
True      692
Name: DPTO, dtype: int64

### Ciudad Principal o Área Metropolitana  
Ciudades Principales y áreas metropolitanas:  
- 11001: Bogotá  
- 5001: Medellín  
    - Valle de Aburra:  
    - 5079: Barbosa  
    - 5088: Bello  
    - 5129: Caldas  
    - 5212: Copacabana  
    - 5266: Envigado  
    - 5308: Girardota  
    - 5360: Itagui  
    - 5380: La Estrella  
    - 5631: Sabaneta  
- 76001: Cali  
    - 76892: Yumbo
- 8001: Barranquilla
    - 8758: Soledad
- 68001: Bucaramanga
    - 68276: Floridablanca
    - 68307: Girón
    - 68547: Piedecuesta
- 17001: Manizales
    - 17873: Villamaría
- 52001: Pasto
- 66001: Pereira
    - 66170: Dosquebradas
    - 66400: La Virginia
- 73001: Ibagué
- 54001: Cúcuta
    - 54874: Villa del Rosario
    - 54405: Los Patios
    - 54261: El Zulia
- 50001: Villavicencio
- 23001: Montería
- 13001: Cartagena
- 15001: Tunja
- 18001: Florencia
- 19001: Popayán
- 20001: Valledupar
- 27001: Quibdó
- 41001: Neiva
- 44001: Riohacha
- 47001: Santa Marta
- 63001: Armenia
- 70001: Sincelejo
- 88001: San Andrés    

In [99]:
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,	8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,	54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	70001,	88001]

In [100]:
df['CPAM'] = (df['MPIO'].isin([cpam,1]) & df['CLASE_x']==1).apply(lambda x: int(x))

### Actividad Económica

In [101]:
rel_g12 = pd.read_excel(R"D:\DANE - Contrato\2024\Operativo Barrido\Data\Diccionario.xlsx", sheet_name="GRUPOS12")

In [102]:
def extract_first_two_digits(x):
    '''
    Definición de una función para extraer los primeros dos dígitos del CIIU para obtener la división
    x: Columna a la que se le aplicará la columna 
    '''
    try:
        # Convert x to string, split at decimal point if present, and extract first two characters
        first_two_digits_str = str(x).split('.')[0][:2]
        # Convert the extracted string to an integer
        first_two_digits_int = int(first_two_digits_str)
        return first_two_digits_int
    except (ValueError, TypeError):
        # Return NaN (or any other value you prefer) if conversion fails
        return np.nan

# Aplicar  the function to the 'CIIU_3' column
df['Division'] = df['CIIUSELECCIONADO'].apply(extract_first_two_digits)


In [103]:
df.shape

(1979, 359)

In [104]:
df = pd.merge(df,rel_g12,on='Division',how="left")

In [105]:
df[['GRUPOS12','Division','CIIUSELECCIONADO']]

,GRUPOS12,Division,CIIUSELECCIONADO
0,4.0,43.0,4311
1,6.0,49.0,4922
2,5.0,47.0,4711
3,5.0,47.0,4719
4,NaN,NaN,NaN
...,...,...,...
1974,5.0,47.0,4711
1975,9.0,77.0,7710
1976,7.0,56.0,5611
1977,11.0,88.0,8810


### RUT

In [106]:
df['IDRUT']

0        SÍ
1        SÍ
2        SÍ
3        SÍ
4       NaN
       ... 
1974     SÍ
1975     SÍ
1976     NO
1977     SÍ
1978     SÍ
Name: IDRUT, Length: 1979, dtype: object

### Cámara de Comercio

In [107]:
# Función para verificar si '1' está presente en la cadena
def contain_one(value):
    '''
    Crear un vector de 1's y 0's con el objetivo crear la variable de Cámara de Comercio
    '''
    if pd.isna(value):
        return 0
    return 1 if '1' in value.split(',') else 0

# Crear la nueva columna
df['CAMCOMER'] = df['CUAL_AUT'].apply(contain_one)

### Registros Contables

In [108]:
df['REG_CONT'] = df['REG_CONT'].map(dict(zip(rel_reg_con['REG_CONT'], rel_reg_con['Codigo'])))

In [109]:
df['REG_CONT'].value_counts()

4.0    623
3.0    513
2.0    262
1.0    182
Name: REG_CONT, dtype: int64

### Antigüedad

In [110]:
df['ANIOS_OPERACION'] = 2024 - df['IDAIO']

### Ubicación

In [111]:
df['UBICA'] = df['UBICA'].map(dict(zip(rel_ubica['UBICA'], rel_ubica['Codigo'])))

## Creación Variables

In [112]:
df['VA'] = df['ING_TOTAL'] - df['CYG']
df['C_V'] = df['CYG'] / (df['ING_TOTAL'] + 1)

## Selección Variables

In [113]:
var_estab = ['UID_ENCUESTA','ING_TOTAL','CYG','POTTOT','REMUNERACION_TOTAL','DPTO','CPAM','GRUPOS12','UBICA','IDRUT','CAMCOMER','REG_CONT','ANIOS_OPERACION','VA','C_V'] 

var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','ANOS_ESPACIO','HOMBRE']

In [114]:
df['UBICA'].value_counts()

1.0    1202
3.0     393
2.0      81
Name: UBICA, dtype: int64

In [115]:
df_final = df[var_estab]

In [116]:
df_final.head()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,SÍ,0,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,SÍ,0,NaN,NaN,NaN,NaN
2,1472675,150000000.0,228720000.0,7.0,15528000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-78720000.0,1.524800
3,4264925,180000000.0,99840000.0,4.0,11136000.0,91.0,1,5.0,1.0,SÍ,1,2.0,27.0,80160000.0,0.554667
4,4780793,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


## Filtrado de unidades

In [117]:
# Seleccionar unidades económicas ubicadas en vivienda o establecimiento fijo (1 y 3)
df_final = df_final[df_final['UBICA'].isin([1,3])]

In [118]:
# Departamentos con los que se entrenó el modelo
depto_emicron = [8, 13, 44, 47, 70, 20, 25, 11, 18, 41, 15, 68, 54, 76, 52, 19, 17, 63, 73, 66,  5, 23, 27, 50, 88]
df_final = df_final[df_final['DPTO'].isin(depto_emicron)]

In [123]:
# Descripción variables nulas
df_final.isna().sum()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
1,False,True,True,True,True,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
14,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1931,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1933,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1954,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [130]:
id_na = df_final['UID_ENCUESTA'][df_final.isna().any(axis=1)]

In [132]:
df_final.head()

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,SÍ,0,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,SÍ,0,NaN,NaN,NaN,NaN
2,1472675,150000000.0,2.287200e+08,7.0,15528000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-7.872000e+07,1.524800
14,43,10100000.0,2.000000e+09,1.0,0.0,11.0,1,5.0,1.0,NO,0,1.0,24.0,-1.989900e+09,198.019782
15,45,1000005.0,1.000050e+05,12.0,48000000.0,11.0,1,7.0,1.0,SÍ,1,1.0,24.0,9.000000e+05,0.100004


In [120]:
# Descripción variables no responde
(df_final==99).sum()

UID_ENCUESTA          0
ING_TOTAL             0
CYG                   0
POTTOT                0
REMUNERACION_TOTAL    0
DPTO                  0
CPAM                  0
GRUPOS12              0
UBICA                 0
IDRUT                 0
CAMCOMER              0
REG_CONT              0
ANIOS_OPERACION       0
VA                    0
C_V                   0
dtype: int64

In [147]:
(df_final['UID_ENCUESTA'].isin(ids))

0       False
1       False
2       False
14       True
15      False
        ...  
1925    False
1931    False
1933    False
1954    False
1968    False
Name: UID_ENCUESTA, Length: 663, dtype: bool

In [148]:
df_final.head(14)

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V
0,72,NaN,NaN,1.0,0.0,11.0,1,4.0,1.0,SÍ,0,3.0,24.0,NaN,NaN
1,74,NaN,NaN,NaN,NaN,11.0,1,6.0,1.0,SÍ,0,NaN,NaN,NaN,NaN
2,1472675,150000000.0,2.287200e+08,7.0,15528000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-7.872000e+07,1.524800
14,43,10100000.0,2.000000e+09,1.0,0.0,11.0,1,5.0,1.0,NO,0,1.0,24.0,-1.989900e+09,198.019782
15,45,1000005.0,1.000050e+05,12.0,48000000.0,11.0,1,7.0,1.0,SÍ,1,1.0,24.0,9.000000e+05,0.100004
16,1463212,500000.0,1.720000e+06,1.0,0.0,13.0,0,3.0,1.0,NO,0,4.0,14.0,-1.220000e+06,3.439993
29,1466730,144000000.0,1.087050e+08,3.0,0.0,13.0,0,5.0,3.0,NO,0,3.0,8.0,3.529500e+07,0.754896
30,1469310,9600000.0,4.760000e+06,1.0,0.0,13.0,0,12.0,1.0,NO,1,4.0,15.0,4.840000e+06,0.495833
31,1469318,54000000.0,6.046800e+07,4.0,0.0,13.0,0,5.0,1.0,SÍ,1,3.0,24.0,-6.468000e+06,1.119778
32,1469412,102000000.0,1.482560e+08,2.0,4200000.0,13.0,0,5.0,1.0,SÍ,1,3.0,5.0,-4.625600e+07,1.453490


In [154]:
df_final['UID_ENCUESTA'][(df_final['UID_ENCUESTA'].isin(ids))]

14     43
218    26
Name: UID_ENCUESTA, dtype: int64

In [160]:
df[['SALFIJOH','SALFIJOM','SALPERMH','SALPERMM','SALPOHTEMP','SALPOMTEMP','MESOPA']][df['UID_ENCUESTA']==70]

,SALFIJOH,SALFIJOM,SALPERMH,SALPERMM,SALPOHTEMP,SALPOMTEMP,MESOPA
35,0.0,0.0,99.0,0.0,0.0,0.0,1.0


In [167]:
df_final[df_final['UID_ENCUESTA']==399]

,UID_ENCUESTA,ING_TOTAL,CYG,POTTOT,REMUNERACION_TOTAL,DPTO,CPAM,GRUPOS12,UBICA,IDRUT,CAMCOMER,REG_CONT,ANIOS_OPERACION,VA,C_V


In [158]:
df_final['UID_ENCUESTA'][df_final['REMUNERACION_TOTAL']==99]

35    70
Name: UID_ENCUESTA, dtype: int64

In [179]:
df['POTTOT'].describe()

count    1130.000000
mean        3.197345
std        11.211273
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       322.000000
Name: POTTOT, dtype: float64

In [180]:
df['POTTOT']

0        1.0
1        NaN
2        7.0
3        4.0
4        NaN
        ... 
1974     1.0
1975     2.0
1976     4.0
1977    17.0
1978     NaN
Name: POTTOT, Length: 1979, dtype: float64